In [1]:
from datetime import datetime, timezone, timedelta
import pandas as pd
from io import StringIO
import import_ipynb
from evaluation import to_timestamp, request_influxdb, query_data, histplots, boxplots, lineplots, missing_data, correlation


importing Jupyter notebook from evaluation.ipynb


In [2]:
# Load Tangaras
tangaras = pd.read_csv('../data/tangaras.csv')
tangaras.shape

(6, 6)

In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#start_timestamp = to_timestamp('2023-03-17T00:00:00-05:00')
#start_timestamp = to_timestamp('2023-04-03T00:00:00-05:00')
start_timestamp = to_timestamp('2023-06-12T00:00:00-05:00')

# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#end_timestamp = to_timestamp('2023-03-31T00:00:00-05:00')
#end_timestamp = to_timestamp('2023-04-04T00:00:00-05:00')
end_timestamp = to_timestamp('2023-06-18T23:59:59-05:00')

print(f'Since: {start_timestamp} Until: {end_timestamp}')


Since: 1686546000000 Until: 1687150799000


In [4]:
# Get Data Frame Sensors
def df_sensors(tangaras, start_timestamp, end_timestamp):
    # Data Frame Sensors
    df_sensors = []
    # SQL Query Data Sensors
    query = query_data(tangaras, start_timestamp, end_timestamp)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(query)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame InfluxDB Sensors
    df_influxdb_sensors = pd.read_csv(StringIO(influxdb_request.text), sep=",", low_memory=False)

    # Remove/Add Columns
    df_influxdb_sensors = df_influxdb_sensors[['time', 'name.1', 'last']]
    df_influxdb_sensors.rename(columns={'time': 'DATETIME', 'name.1': 'MAC', 'last': 'PM25'}, inplace=True)

    # Truncate Response
    for index, row in tangaras.iterrows():
        df_sensor = df_influxdb_sensors.loc[df_influxdb_sensors['MAC'] == row['MAC']].reset_index(drop=True)[['DATETIME', 'PM25']] # Warning
        if not df_sensor.empty:
            df_sensor.rename(columns={'PM25': row['ID']}, inplace=True)
            df_sensor.set_index('DATETIME', inplace=True)
            df_sensors.append(df_sensor)
    
    df_sensors = df_sensors[0].join(df_sensors[1:]).reset_index()

    # Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
    tz = timezone(timedelta(hours=-5))
    df_sensors['DATETIME'] = df_sensors['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz).isoformat())
    df_sensors['DATETIME'] = pd.to_datetime(df_sensors['DATETIME'])

    # Set Index
    df_sensors.set_index('DATETIME', inplace=True)
    df_sensors = df_sensors.asfreq(freq='30S')

    # Data type as float64
    df_sensors[df_sensors.columns.to_list()] = df_sensors[df_sensors.columns.to_list()].astype('float64')
    
    return df_sensors


In [5]:
# Data Frame Sensors
pm25_raw = df_sensors(tangaras, start_timestamp, end_timestamp)
pm25_raw.head()

,TANGARA_260A,TANGARA_1282,TANGARA_2492,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,
2023-06-12 00:00:30-05:00,8.0,5.0,8.0,3.0,3.0,10.0
2023-06-12 00:01:00-05:00,8.0,9.0,9.0,5.0,3.0,10.0
2023-06-12 00:01:30-05:00,8.0,7.0,9.0,6.0,3.0,11.0
2023-06-12 00:02:00-05:00,8.0,8.0,9.0,7.0,3.0,10.0
2023-06-12 00:02:30-05:00,9.0,8.0,10.0,3.0,2.0,10.0


# Descriptive Statistics

In [6]:
# Describe Data
pm25_raw.describe()

,TANGARA_260A,TANGARA_1282,TANGARA_2492,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,18995.000000,18101.000000,18992.000000,18095.000000,17271.000000,18655.000000
mean,9.026112,10.422794,8.672336,7.339376,7.688206,10.661431
std,4.997183,6.951972,18.425962,4.878466,3.844126,7.536536
min,2.000000,1.000000,1.000000,0.000000,2.000000,2.000000
25%,6.000000,6.000000,6.000000,4.000000,5.000000,7.000000
50%,8.000000,9.000000,7.000000,6.000000,7.000000,9.000000
75%,11.000000,13.000000,9.000000,10.000000,10.000000,13.000000
max,45.000000,87.000000,922.000000,76.000000,37.000000,495.000000


In [7]:
# Missing Data Sensors
threshold = 85
to_be_checked = missing_data(pm25_raw, threshold)

print(f'Threshold: {threshold}%')
print(f'Total Sensors: {len(pm25_raw.columns)}')
print(f'To Be Checked: {len(to_be_checked)}')
to_be_checked


Threshold: 85%
Total Sensors: 6
To Be Checked: 0


[]

In [8]:
# Sensors to evaluate by Pearson correlation coefficient
sensors_to_evaluate = pm25_raw[['TANGARA_1282', 'TANGARA_260A', 'TANGARA_06BE']]

# Pearson correlation coefficient
threshold = 0.85
to_be_checked = correlation(sensors_to_evaluate, 'TANGARA_1282', threshold)

# Correlation Sensors
print(f'Threshold: {threshold}')
print(f'Total Sensors: {len(sensors_to_evaluate.columns)}')
print(f'To Be Checked: {len(to_be_checked)}')
to_be_checked


Threshold: 0.85
Total Sensors: 3
To Be Checked: 2


[{'Reference': {'ID': 'TANGARA_1282', 'Data': 20157, 'Freq': <30 * Seconds>},
  'Target': {'ID': 'TANGARA_260A', 'Data': 20157, 'Freq': <30 * Seconds>},
  'Correlation': 0.7775261855191344},
 {'Reference': {'ID': 'TANGARA_1282', 'Data': 20157, 'Freq': <30 * Seconds>},
  'Target': {'ID': 'TANGARA_06BE', 'Data': 20157, 'Freq': <30 * Seconds>},
  'Correlation': 0.3418650799386166}]

In [9]:
# Save PM25 raw data into CSV file
pm25_raw.to_csv('../data/pm25_raw.csv')